In [14]:
import random
from typing import List, Dict, Set, Tuple
import numpy as np
from alns import ALNS, State
from alns.accept import SimulatedAnnealing
from alns.select import RouletteWheel
from alns.stop import MaxIterations


class Route:
    """
    Represents a van or robot route in the 2E-VREC problem.
    """

    def __init__(self, is_van_route: bool = True):
        self.nodes: List[int] = []  # List of nodes in the route
        self.is_van_route: bool = is_van_route
        # Whether robot is on the van for each segment (van routes only)
        self.robot_onboard: List[bool] = []
        # Amount to recharge at each node
        self.recharge_amount: Dict[int, float] = {}
        # Amount to charge en-route (van routes only)
        self.en_route_charge: Dict[Tuple[int, int], float] = {}

    def copy(self) -> "Route":
        """Creates a deep copy of the VehicleRoute object."""
        route = Route(self.is_van_route)
        route.nodes = self.nodes.copy()
        route.robot_onboard = self.robot_onboard.copy()
        route.recharge_amount = self.recharge_amount.copy()
        route.en_route_charge = self.en_route_charge.copy()
        return route

    def __str__(self) -> str:
        route_type = "Van" if self.is_van_route else "Robot"
        return f"{route_type} Route: {self.nodes}"


class TwoEVREC(State):
    """
    Represents a solution to the 2E-VREC problem.
    """

    def __init__(self,
                 distance_matrix: np.ndarray,
                 depot: int,
                 charging_stations: List[int],
                 customers_robot_only: List[int],
                 customers_both: List[int],
                 van_params: Dict[str, float],
                 robot_params: Dict[str, float],
                 customer_demand: Dict[int, float]):

        # Problem data
        self.distance_matrix: np.ndarray = distance_matrix
        self.depot: int = depot
        self.charging_stations: List[int] = charging_stations
        self.customers_robot_only: List[int] = customers_robot_only
        self.customers_both: List[int] = customers_both
        self.all_customers: List[int] = customers_robot_only + customers_both
        self.van_params: Dict[str, float] = van_params
        self.robot_params: Dict[str, float] = robot_params
        self.customer_demand: Dict[int, float] = customer_demand

        # Solution data
        self.van_routes: List[Route] = []
        self.robot_routes: List[Route] = []
        self.unassigned_customers: Set[int] = set(self.all_customers)

    def copy(self) -> "TwoEVREC":
        """Create a deep copy of the solution state."""
        solution = TwoEVREC(
            self.distance_matrix,
            self.depot,
            self.charging_stations,
            self.customers_robot_only,
            self.customers_both,
            self.van_params,
            self.robot_params,
            self.customer_demand
        )

        solution.van_routes = [route.copy() for route in self.van_routes]
        solution.robot_routes = [route.copy() for route in self.robot_routes]
        solution.unassigned_customers = self.unassigned_customers.copy()

        return solution

    def objective(self) -> float:
        """
        Calculate the total cost of the solution.  This includes the van
        route cost plus the robot route cost, minus the van-robot route cost.
        """
        total_cost = 0.0

        # Van route cost
        for route in self.van_routes:
            if len(route.nodes) > 1:  # Only calculate if route has at least 2 nodes
                for i in range(len(route.nodes) - 1):
                    node1, node2 = route.nodes[i], route.nodes[i + 1]
                    total_cost += (self.van_params["travel_cost_rate"] *
                                   self.distance_matrix[node1][node2])

        # Robot route cost
        for route in self.robot_routes:
            if len(route.nodes) > 1:
                for i in range(len(route.nodes) - 1):
                    node1, node2 = route.nodes[i], route.nodes[i + 1]
                    total_cost += (self.robot_params["travel_cost_rate"] *
                                   self.distance_matrix[node1][node2])

        # Subtract van-robot route cost (where robot is onboard)
        for route in self.van_routes:
            if len(route.nodes) > 1:
                for i in range(len(route.nodes) - 1):
                    if i < len(route.robot_onboard) and route.robot_onboard[i]:
                        node1, node2 = route.nodes[i], route.nodes[i + 1]
                        # Subtract robot cost (robot not moving independently)
                        total_cost -= (self.robot_params["travel_cost_rate"] *
                                       self.distance_matrix[node1][node2])

        return total_cost

    def is_feasible(self) -> bool:
        """
        Check if the solution is feasible:
            - All customers are served
            - Vehicle capacities are respected
            - Energy constraints are respected
            - Every robot can complete its route given battery constraints
        """
        if self.unassigned_customers:
            return False

        return all(self._is_van_route_feasible(route, i)
                   for i, route in enumerate(self.van_routes)) and \
               all(self._is_robot_route_feasible(route, i)
                   for i, route in enumerate(self.robot_routes))

    def _is_van_route_feasible(self, route: Route,
                               route_idx: int) -> bool:
        """Check if a van route is feasible."""
        if not route.nodes:
            return True  # Empty route is feasible

        if route.nodes[0] != self.depot or route.nodes[-1] != self.depot:
            return False

        battery_level = self.van_params["battery_capacity"]
        load = 0

        for i in range(len(route.nodes) - 1):
            node1, node2 = route.nodes[i], route.nodes[i + 1]

            if node1 in self.all_customers:
                load -= self.customer_demand[node1]  # Delivered, reduce load

            distance = self.distance_matrix[node1][node2]
            energy_consumption = (distance *
                                  self.van_params["energy_consumption_rate"])
            en_route_charge = route.en_route_charge.get((node1, node2), 0)

            if node1 in self.charging_stations:
                recharge_amount = route.recharge_amount.get(node1, 0)
                battery_level = min(battery_level + recharge_amount,
                                    self.van_params["battery_capacity"])

            if battery_level < energy_consumption + en_route_charge:
                return False

            battery_level -= energy_consumption + en_route_charge

        return load <= self.van_params["capacity"]

    def _is_robot_route_feasible(self, route: Route,
                                 route_idx: int) -> bool:
        """Check if a robot route is feasible."""
        if not route.nodes:
            return True

        if (route.nodes[0] not in self.charging_stations or
                route.nodes[-1] not in self.charging_stations):
            return False

        battery_level = route.recharge_amount.get(route.nodes[0], 0)
        load = 0

        for i in range(len(route.nodes) - 1):
            node1, node2 = route.nodes[i], route.nodes[i + 1]

            if node1 in self.all_customers:
                load -= self.customer_demand[node1]

            distance = self.distance_matrix[node1][node2]
            energy_consumption = (distance *
                                  self.robot_params["energy_consumption_rate"])

            if node1 in self.charging_stations:
                recharge_amount = route.recharge_amount.get(node1, 0)
                battery_level = min(battery_level + recharge_amount,
                                    self.robot_params["battery_capacity"])

            if battery_level < energy_consumption:
                return False

            battery_level -= energy_consumption

        return load <= self.robot_params["capacity"]

    def _can_van_visit_customer(self, route: Route, customer: int) -> bool:
        """Check if a van can visit a customer directly."""
        if customer in self.customers_robot_only:
            return False

        return self.customer_demand[customer] <= self.van_params["capacity"]

    def create_initial_solution(self) -> "TwoEVREC":
        """
        Create an initial feasible solution.  Uses a simple greedy approach
        to assign customers to routes.
        """
        solution = self.copy()
        solution.unassigned_customers = set(self.all_customers)

        # Create a simple solution: one van route, one robot route
        van_route = Route(is_van_route=True)
        van_route.nodes = [self.depot]

        # Closest charging station to the depot
        station_distances = [(s, self.distance_matrix[self.depot][s])
                             for s in self.charging_stations]
        station_distances.sort(key=lambda x: x[1])
        closest_station = station_distances[0][0]

        van_route.nodes.append(closest_station)
        van_route.robot_onboard = [True]  # Robot on board from depot to station

        # Create robot route starting/ending at this station
        robot_route = Route(is_van_route=False)
        robot_route.nodes = [closest_station]

        customers_to_visit = list(solution.unassigned_customers)
        customers_to_visit.sort(
            key=lambda c: self.distance_matrix[closest_station][c])

        total_load = 0
        for customer in customers_to_visit:
            if customer in solution.customers_both:
                # Van can visit, randomly choose van or robot
                if (random.random() < 0.5 and
                        self._can_van_visit_customer(van_route, customer)):
                    van_route.nodes.insert(-1, customer)  # Insert before station
                    van_route.robot_onboard.append(True)  # Robot stays onboard
                    solution.unassigned_customers.remove(customer)
                    continue

            # Try to add to robot route
            if (total_load + self.customer_demand[customer] <=
                    self.robot_params["capacity"]):
                robot_route.nodes.append(customer)
                total_load += self.customer_demand[customer]
                solution.unassigned_customers.remove(customer)

        # Complete routes
        robot_route.nodes.append(closest_station)
        van_route.nodes.append(self.depot)
        van_route.robot_onboard.append(True)  # Robot onboard: station to depot

        # Set recharging values
        van_route.recharge_amount[closest_station] = (
            self.van_params["battery_capacity"] / 2)  # Recharge van
        robot_route.recharge_amount[closest_station] = (
            self.robot_params["battery_capacity"])  # Fully charge robot

        solution.van_routes.append(van_route)
        solution.robot_routes.append(robot_route)

        # If unassigned customers, create more routes
        while solution.unassigned_customers:
            new_van_route = Route(is_van_route=True)
            new_van_route.nodes = [self.depot, closest_station, self.depot]
            new_van_route.robot_onboard = [True, True]  # Robot onboard

            new_robot_route = Route(is_van_route=False)
            new_robot_route.nodes = [closest_station]

            customers_to_visit = list(solution.unassigned_customers)
            customers_to_visit.sort(
                key=lambda c: self.distance_matrix[closest_station][c])

            total_load = 0
            for customer in customers_to_visit:
                if (total_load + self.customer_demand[customer] <=
                        self.robot_params["capacity"]):
                    new_robot_route.nodes.append(customer)
                    total_load += self.customer_demand[customer]
                    solution.unassigned_customers.remove(customer)

            if len(new_robot_route.nodes) > 1:  # Added at least one customer
                new_robot_route.nodes.append(closest_station)
                new_robot_route.recharge_amount[closest_station] = (
                    self.robot_params["battery_capacity"])
                new_van_route.recharge_amount[closest_station] = (
                    self.van_params["battery_capacity"] / 2)
                solution.van_routes.append(new_van_route)
                solution.robot_routes.append(new_robot_route)
            else:
                break  # Couldn't add any customers

        return solution


# Destroy operators
def random_customer_removal(solution: TwoEVREC,
                            random_state: np.random.RandomState) -> TwoEVREC:
    """
    Remove a random customer from the solution.
    """
    destroyed = solution.copy()
    assigned_customers = [c for c in destroyed.all_customers
                          if c not in destroyed.unassigned_customers]

    if not assigned_customers:
        return destroyed  # No customers to remove

    customer_to_remove = random_state.choice(assigned_customers)

    for route in destroyed.van_routes:
        if customer_to_remove in route.nodes:
            idx = route.nodes.index(customer_to_remove)
            route.nodes.pop(idx)
            if idx < len(route.robot_onboard):
                route.robot_onboard.pop(idx)

    for route in destroyed.robot_routes:
        if customer_to_remove in route.nodes:
            route.nodes.remove(customer_to_remove)

    destroyed.unassigned_customers.add(customer_to_remove)
    return destroyed


def station_removal(solution: TwoEVREC,
                    random_state: np.random.RandomState) -> TwoEVREC:
    """
    Remove a random charging station and its associated robot routes.
    """
    destroyed = solution.copy()
    used_stations = set()
    for route in destroyed.van_routes:
        for node in route.nodes:
            if node in destroyed.charging_stations:
                used_stations.add(node)

    if not used_stations:
        return destroyed  # No stations to remove

    station_to_remove = random_state.choice(list(used_stations))

    routes_to_remove = []
    for i, route in enumerate(destroyed.robot_routes):
        if station_to_remove in route.nodes:
            routes_to_remove.append(i)
            for node in route.nodes:
                if node in destroyed.all_customers:
                    destroyed.unassigned_customers.add(node)

    for i in sorted(routes_to_remove, reverse=True):
        destroyed.robot_routes.pop(i)

    for route in destroyed.van_routes:
        if station_to_remove in route.nodes:
            idx = route.nodes.index(station_to_remove)
            if 0 < idx < len(route.nodes) - 1:
                route.nodes.pop(idx)
                if idx < len(route.robot_onboard):
                    route.robot_onboard.pop(idx)

    return destroyed


def route_destruction(solution: TwoEVREC,
                      random_state: np.random.RandomState) -> TwoEVREC:
    """
    Destroy a random van route and its associated robot routes.
    """
    destroyed = solution.copy()
    if not destroyed.van_routes:
        return destroyed  # No routes to destroy

    van_route_idx = random_state.randint(0, len(destroyed.van_routes))

    if van_route_idx < len(destroyed.van_routes):
        van_route = destroyed.van_routes[van_route_idx]

        for node in van_route.nodes:
            if node in destroyed.all_customers:
                destroyed.unassigned_customers.add(node)

        stations_in_route = [node for node in van_route.nodes
                             if node in destroyed.charging_stations]

        routes_to_remove = []
        for i, route in enumerate(destroyed.robot_routes):
            if any(station in route.nodes for station in stations_in_route):
                routes_to_remove.append(i)
                for node in route.nodes:
                    if node in destroyed.all_customers:
                        destroyed.unassigned_customers.add(node)

        for i in sorted(routes_to_remove, reverse=True):
            destroyed.robot_routes.pop(i)

        destroyed.van_routes.pop(van_route_idx)

    return destroyed


# Repair operators
def greedy_repair(solution: TwoEVREC,
                  random_state: np.random.RandomState) -> TwoEVREC:
    """
    Repair by greedily inserting unassigned customers into existing routes.
    """
    repaired = solution.copy()
    unassigned = list(repaired.unassigned_customers)
    random_state.shuffle(unassigned)

    for customer in unassigned:
        best_cost = float('inf')
        best_insert = None

        # Try to insert into van routes
        if customer in repaired.customers_both:
            for r_idx, route in enumerate(repaired.van_routes):
                for pos in range(1, len(route.nodes)):
                    prev_node, next_node = route.nodes[pos - 1], route.nodes[pos]
                    old_cost = (repaired.van_params["travel_cost_rate"] *
                                repaired.distance_matrix[prev_node][next_node])
                    new_cost = (repaired.van_params["travel_cost_rate"] *
                                (repaired.distance_matrix[prev_node][customer] +
                                 repaired.distance_matrix[customer][next_node]))
                    delta = new_cost - old_cost

                    if delta < best_cost:
                        temp_route = route.copy()
                        temp_route.nodes.insert(pos, customer)
                        temp_route.robot_onboard.insert(
                            pos - 1,
                            (route.robot_onboard[pos - 1]
                             if pos - 1 < len(route.robot_onboard) else False)
                        )

                        if repaired._is_van_route_feasible(temp_route, r_idx):
                            best_cost = delta
                            best_insert = ('van', r_idx, pos)

        # Try to insert into robot routes
        for r_idx, route in enumerate(repaired.robot_routes):
            for pos in range(1, len(route.nodes)):
                prev_node, next_node = route.nodes[pos - 1], route.nodes[pos]
                old_cost = (repaired.robot_params["travel_cost_rate"] *
                            repaired.distance_matrix[prev_node][next_node])
                new_cost = (repaired.robot_params["travel_cost_rate"] *
                            (repaired.distance_matrix[prev_node][customer] +
                             repaired.distance_matrix[customer][next_node]))
                delta = new_cost - old_cost

                if delta < best_cost:
                    temp_route = route.copy()
                    temp_route.nodes.insert(pos, customer)

                    if repaired._is_robot_route_feasible(temp_route, r_idx):
                        best_cost = delta
                        best_insert = ('robot', r_idx, pos)

        if best_insert:
            route_type, r_idx, pos = best_insert
            if route_type == 'van':
                repaired.van_routes[r_idx].nodes.insert(pos, customer)
                repaired.van_routes[r_idx].robot_onboard.insert(
                    pos - 1,
                    (repaired.van_routes[r_idx].robot_onboard[pos - 1]
                     if pos - 1 < len(repaired.van_routes[r_idx].robot_onboard)
                     else False)
                )
            else:  # robot
                repaired.robot_routes[r_idx].nodes.insert(pos, customer)
            repaired.unassigned_customers.remove(customer)
        else:
            # Create a new route (van or robot)
            if customer in repaired.customers_both:
                new_van_route = Route(is_van_route=True)
                new_van_route.nodes = [repaired.depot, customer, repaired.depot]
                new_van_route.robot_onboard = [False, False]

                if repaired._is_van_route_feasible(new_van_route,
                                                  len(repaired.van_routes)):
                    repaired.van_routes.append(new_van_route)
                    repaired.unassigned_customers.remove(customer)
                    continue

            # Need to use a robot, choose closest charging station
            station_distances = [
                (s, repaired.distance_matrix[customer][s])
                for s in repaired.charging_stations
            ]
            station_distances.sort(key=lambda x: x[1])
            closest_station = station_distances[0][0]

            new_van_route = Route(is_van_route=True)
            new_van_route.nodes = [repaired.depot, closest_station, repaired.depot]
            new_van_route.robot_onboard = [True, True]
            new_van_route.recharge_amount[closest_station] = (
                repaired.van_params["battery_capacity"] / 2)

            new_robot_route = Route(is_van_route=False)
            new_robot_route.nodes = [closest_station, customer, closest_station]
            new_robot_route.recharge_amount[closest_station] = (
                repaired.robot_params["battery_capacity"])

            if (repaired._is_van_route_feasible(new_van_route,
                                                len(repaired.van_routes)) and
                    repaired._is_robot_route_feasible(new_robot_route,
                                                     len(repaired.robot_routes))):
                repaired.van_routes.append(new_van_route)
                repaired.robot_routes.append(new_robot_route)
                repaired.unassigned_customers.remove(customer)

    return repaired


def route_reconstruction(solution: TwoEVREC,
                         random_state: np.random.RandomState) -> TwoEVREC:
    """
    Reconstruct routes for all unassigned customers.
    """
    reconstructed = solution.copy()
    if not reconstructed.unassigned_customers:
        return reconstructed

    unassigned = list(reconstructed.unassigned_customers)
    unassigned.sort(key=lambda c: reconstructed.customer_demand[c],
                    reverse=True)  # Sort by demand (descending)

    for customer in unassigned:
        if (customer in reconstructed.customers_both and
                random_state.random() < 0.5):
            # Try to use van directly
            new_van_route = Route(is_van_route=True)
            new_van_route.nodes = [reconstructed.depot, customer,
                                   reconstructed.depot]
            new_van_route.robot_onboard = [False, False]

            if reconstructed._is_van_route_feasible(
                    new_van_route, len(reconstructed.van_routes)):
                reconstructed.van_routes.append(new_van_route)
                reconstructed.unassigned_customers.remove(customer)
                continue

        # Need to use a robot, find closest station
        station_distances = [
            (s, reconstructed.distance_matrix[customer][s])
            for s in reconstructed.charging_stations
        ]
        station_distances.sort(key=lambda x: x[1])

        for closest_station, _ in station_distances:
            new_van_route = Route(is_van_route=True)
            new_van_route.nodes = [reconstructed.depot, closest_station,
                                   reconstructed.depot]
            new_van_route.robot_onboard = [True, True]
            new_van_route.recharge_amount[closest_station] = (
                reconstructed.van_params["battery_capacity"] / 2)

            new_robot_route = Route(is_van_route=False)
            new_robot_route.nodes = [closest_station, customer, closest_station]
            new_robot_route.recharge_amount[closest_station] = (
                reconstructed.robot_params["battery_capacity"])

            if (reconstructed._is_van_route_feasible(
                    new_van_route, len(reconstructed.van_routes)) and
                    reconstructed._is_robot_route_feasible(
                        new_robot_route, len(reconstructed.robot_routes))):
                reconstructed.van_routes.append(new_van_route)
                reconstructed.robot_routes.append(new_robot_route)
                reconstructed.unassigned_customers.remove(customer)
                break  # Added, move to next customer

    return reconstructed


def main():
    """Main function to run the ALNS algorithm."""
    distance_matrix = np.array([
        [0, 50, 80, 80, 50, 80, 80, 82, 84, 86],  # depot 0
        [50, 0, 0, 80, 80, 80, 50, 50, 90, 90],  # station 1
        [80, 70, 0, 50, 80, 80, 80, 80, 80, 80],  # station 2
        [80, 80, 50, 0, 50, 80, 50, 50, 80, 80],  # station 3
        [50, 80, 80, 50, 0, 70, 90, 80, 30, 30],  # station 4
        [80, 80, 80, 80, 70, 0, 80, 80, 80, 80],  # station 5
        [80, 50, 80, 50, 90, 80, 0, 80, 90, 90],  # customer 6
        [82, 50, 80, 50, 80, 80, 80, 0, 90, 90],  # customer 7
        [84, 90, 80, 80, 30, 80, 90, 90, 0, 30],  # customer 8
        [86, 90, 80, 80, 30, 80, 90, 90, 30, 0]  # customer 9
    ])
    nodes = list(range(len(distance_matrix)))  # Not needed
    depot = 0
    charging_stations = [1, 2, 3, 4, 5]
    customers_robot_only = [6, 8, 9]
    customers_both = [7]
    all_customers = customers_robot_only + customers_both  # Not needed

    van_params = {
        "speed": 2.0,
        "battery_capacity": 400.0,
        "capacity": 200.0,
        "travel_cost_rate": 2.0,
        "energy_consumption_rate": 2.0,
        "charge_rate": 10.0
    }

    robot_params = {
        "speed": 1.0,
        "battery_capacity": 120.0,
        "capacity": 50.0,
        "travel_cost_rate": 1.0,
        "energy_consumption_rate": 1.0,
        "charge_rate": 4.0
    }

    customer_demand = {
        6: 15,
        7: 20,
        8: 10,
        9: 25
    }

    problem = TwoEVREC(
        distance_matrix=distance_matrix,
        depot=depot,
        charging_stations=charging_stations,
        customers_robot_only=customers_robot_only,
        customers_both=customers_both,
        van_params=van_params,
        robot_params=robot_params,
        customer_demand=customer_demand
    )

    initial_solution = problem.create_initial_solution()

    alns = ALNS()

    alns.add_destroy_operator(random_customer_removal)
    alns.add_destroy_operator(station_removal)
    alns.add_destroy_operator(route_destruction)

    alns.add_repair_operator(greedy_repair)
    alns.add_repair_operator(route_reconstruction)

    select = RouletteWheel([25, 5, 1, 0], 0.8, 1, 1)
    accept = SimulatedAnnealing(
        start_temperature=100,
        end_temperature=1,
        step=0.99,
        method="exponential"
    )
    stop = MaxIterations(1000)

    result = alns.iterate(initial_solution, select, accept, stop)
    best_solution = result.best_state

    print(f"Best solution objective value: {best_solution.objective()}")

    print("\nVan Routes:")
    for i, route in enumerate(best_solution.van_routes):
        print(f"  Route {i + 1}: {route.nodes}")
        print(f"    Robot onboard: {route.robot_onboard}")
        print(f"    Recharge amounts: {route.recharge_amount}")
        print(f"    En-route charging: {route.en_route_charge}")

    print("\nRobot Routes:")
    for i, route in enumerate(best_solution.robot_routes):
        print(f"  Route {i + 1}: {route.nodes}")
        print(f"    Recharge amounts: {route.recharge_amount}")

    print(f"\nUnassigned customers: {best_solution.unassigned_customers}")


if __name__ == "__main__":
    main()

Best solution objective value: 442.0

Van Routes:
  Route 1: [0, 7, 1, 0]
    Robot onboard: [True, True, True]
    Recharge amounts: {1: 200.0}
    En-route charging: {}

Robot Routes:
  Route 1: [1, 6, 8, 9, 1]
    Recharge amounts: {1: 120.0}

Unassigned customers: set()
